In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/digit-recognizer/sample_submission.csv
/kaggle/input/digit-recognizer/train.csv
/kaggle/input/digit-recognizer/test.csv


In [2]:
train=pd.read_csv("/kaggle/input/digit-recognizer/train.csv")
test = pd.read_csv("/kaggle/input/digit-recognizer/test.csv")


In [3]:
train.head()
print(train.shape)

(42000, 785)


In [4]:
test.head()


,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torch.optim as optim
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm 
import torchvision
!pip install torchsummary
from torchsummary import summary

In [6]:
features = train.drop(columns=['label']).values
# normalization (0-1)
features = features / 255.0 
print(features.shape)


(42000, 784)


In [7]:
labels = train['label'].values
print(labels.shape)

(42000,)


In [8]:
features = features.reshape(-1,1,28,28)
print("features shape: ",features.shape)

features shape:  (42000, 1, 28, 28)


In [9]:
# converting to torch
features_tensor = torch.tensor(features, dtype=torch.float32)
labels_tensor = torch.tensor(labels, dtype=torch.long)

In [10]:
features_tensor = torch.tensor(features, dtype=torch.float32)
labels_tensor = torch.tensor(labels, dtype=torch.long)

# Create a TensorDataset(every column is a feature)
train_dataset = torch.utils.data.TensorDataset(features_tensor, labels_tensor)
train_loader=DataLoader(dataset=train_dataset,batch_size=64,shuffle=True)

In [11]:
class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.conv1 = nn.Conv2d(1, 4, 3)
        self.bn1 = nn.BatchNorm2d(4)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(4, 8, 3)
        self.bn2 = nn.BatchNorm2d(8)
        self.fc = nn.Linear(8 * 11 * 11, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.bn1(x)
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.bn2(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x
    
model=Network()
input_size=(1,28,28)
summary(model,input_size=input_size)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 4, 26, 26]              40
       BatchNorm2d-2            [-1, 4, 26, 26]               8
         MaxPool2d-3            [-1, 4, 13, 13]               0
            Conv2d-4            [-1, 8, 11, 11]             296
       BatchNorm2d-5            [-1, 8, 11, 11]              16
            Linear-6                   [-1, 10]           9,690
Total params: 10,050
Trainable params: 10,050
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.06
Params size (MB): 0.04
Estimated Total Size (MB): 0.10
----------------------------------------------------------------


In [12]:
learning_rate=3e-4
num_epochs=10
loss_fn=nn.CrossEntropyLoss()
optimizer=optim.Adam(model.parameters(),lr=learning_rate)

In [13]:
for epoch in range(num_epochs):
    for batch_idx, (data, targets) in enumerate(tqdm(train_loader)):
        data = data.view(-1, 1, 28, 28)

        # forward
        scores = model(data)
        loss = loss_fn(scores, targets)

        # backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:

            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

    model.train()
    return num_correct / num_samples


print(f"Accuracy on training set: {check_accuracy(train_loader, model)*100:.2f}")

100%|██████████| 657/657 [00:06<00:00, 98.62it/s] 


Accuracy on training set: 99.21


In [14]:
features_test = test.values.reshape(-1, 1, 28, 28)

test_tensor = torch.tensor(features_test, dtype=torch.float32) / 255.0

model.eval()  
with torch.no_grad():
    test_predictions = model(test_tensor)

model.train() 

predicted_labels = torch.argmax(test_predictions, dim=1).numpy()

submission = pd.DataFrame({"ImageId": range(1, len(predicted_labels) + 1), "Label": predicted_labels})
submission.to_csv("submission0.csv", index=False)

In [15]:
submission.head()

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,9
4,5,3
